<a href="https://colab.research.google.com/github/shgupta1461/Big-Data-Lab/blob/main/BDP_Assignment4_202318052.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   **Name:** Shubham Gupta
*   **Student Id:** 202318052

**BIG DATA PROCEESSING (Assignment 4)**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=2cacebdc782c4e62e97afa7d52c1b743d7d9eea00dc000c7269a95a8c85de339
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import numpy as np

## Create SparkSession

In [5]:
spark = SparkSession.builder \
      .appName("PySpark Session 1") \
      .getOrCreate()

## RDD Task 1

In [6]:
# Generate 100 random numbers using randint()
np.random.seed(10)
random_numbers = np.random.randint(0, 11, size=100)

In [7]:
rdd1 = spark.sparkContext.parallelize(random_numbers)

# Calculate frequency of each number (0-10)
frequency_rdd1 = rdd1.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

print(frequency_rdd1.collect())

[(4, 8), (0, 12), (10, 8), (8, 14), (6, 11), (2, 8), (9, 12), (1, 11), (3, 6), (5, 5), (7, 5)]


## RDD Task 2

In [8]:
# Loading dataset
rdd2 = spark.sparkContext.textFile("/content/text8")

# Split each line into words and get word frequencies
word_frequencies = rdd2.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

# Filter operation to get frequencies of words containing the letter 'a'
word_frequencies_with_a = word_frequencies.filter(lambda x: 'a' in x[0])

print(word_frequencies_with_a.collect())

[('regimentation', 7), ('american', 20477), ('supplant', 24), ('gain', 939), ('dogmatic', 80), ('republican', 842), ('vulnerabilities', 37), ('play', 3702), ('across', 2222), ('ranchers', 5), ('handwriting', 79), ('logicians', 30), ('captured', 814), ('warriors', 237), ('placed', 1685), ('argonautica', 8), ('legal', 2562), ('setbacks', 38), ('virtual', 761), ('answered', 170), ('contain', 1295), ('nanosecond', 5), ('obligations', 203), ('canucks', 5), ('inspiration', 396), ('tags', 95), ('gases', 331), ('capabilities', 352), ('minimal', 339), ('bilingual', 122), ('deregulating', 5), ('aloga', 1), ('restrainer', 1), ('simultaneously', 429), ('nalbandian', 3), ('boldface', 16), ('natufian', 10), ('hausa', 40), ('mania', 120), ('accident', 432), ('blame', 137), ('jeddah', 10), ('snowball', 33), ('gollancz', 26), ('rappers', 81), ('fran', 446), ('waterways', 138), ('overshadowed', 64), ('disposable', 50), ('para', 90), ('thagard', 4), ('culinary', 68), ('syllable', 302), ('bam', 20), ('ali

## DataFrame Task

In [9]:
# schema for iris json data
schema = StructType([
    StructField("sepalLength", FloatType(), True),
    StructField("sepalWidth", FloatType(), True),
    StructField("petalLength", FloatType(), True),
    StructField("petalWidth", FloatType(), True),
    StructField("species", StringType(), True)
])

# Read iris json data into DataFrame
iris_df = spark.read.json("/content/iris.json", schema=schema)

# Calculate Pearson Correlation between petalLength and petalWidth
correlation = iris_df.stat.corr("petalLength", "petalWidth")

# filter rows where petalLength >= 1.4
filtered_df = iris_df.filter(col("petalLength") >= 1.4).select("sepalLength", "sepalWidth", "species")

print("Pearson Correlation between petalLength and petalWidth:", correlation)
filtered_df.show()

Pearson Correlation between petalLength and petalWidth: 0.9626417198678409
+-----------+----------+-------+
|sepalLength|sepalWidth|species|
+-----------+----------+-------+
|        4.6|       3.1| setosa|
|        5.4|       3.9| setosa|
|        5.0|       3.4| setosa|
|        4.9|       3.1| setosa|
|        5.4|       3.7| setosa|
|        4.8|       3.4| setosa|
|        5.7|       4.4| setosa|
|        5.7|       3.8| setosa|
|        5.1|       3.8| setosa|
|        5.4|       3.4| setosa|
|        5.1|       3.7| setosa|
|        5.1|       3.3| setosa|
|        4.8|       3.4| setosa|
|        5.0|       3.0| setosa|
|        5.0|       3.4| setosa|
|        5.2|       3.5| setosa|
|        4.7|       3.2| setosa|
|        4.8|       3.1| setosa|
|        5.4|       3.4| setosa|
|        5.2|       4.1| setosa|
+-----------+----------+-------+
only showing top 20 rows



## Stop SparkSession

In [10]:
spark.stop()